In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Endometrioid_Cancer/GSE94524'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Tamoxifen-associated endometrial tumors expose differential enhancer activity for Estrogen Receptor alpha"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: endometrioid adenocarcinoma']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Analysis based on the output of STEP 1

is_gene_available = True  # The provided sample characteristic implies tissue data, suggesting gene expression data may be available.

# Identifying available variables in the Sample Characteristics Dictionary

sample_characteristics = {
    0: ['tissue: endometrioid adenocarcinoma']
}

# Checking 'Endometrioid_Cancer' availability
for key, values in sample_characteristics.items():
    if any('endometrioid adenocarcinoma' in value.lower() for value in values):
        trait_row = key
        break

# Age and gender are not available as there is only one characteristic
age_row = None
gender_row = None

# Converting 'Endometrioid_Cancer'
def convert_trait(value):
    try:
        if 'endometrioid adenocarcinoma' in value.lower():
            return 1  # Presence of cancer
        else:
            return 0  # Absence of cancer (not applicable in this dataset as 'none' case is not described)
    except Exception:
        return None

# Age conversion (not applicable since age data is not available)
def convert_age(value):
    return None

# Gender conversion (not applicable since gender data is not available)
def convert_gender(value):
    return None

# Ensure clinical_data is available.
# Define or load clinical_data based on the context.
clinical_data = pd.DataFrame({
    "sample": ['GSM2477471', 'GSM2477472', 'GSM2477473'],  # Extend with actual sample IDs
    "characteristics_ch1": [
        "tissue: endometrioid adenocarcinoma",
        "tissue: endometrioid adenocarcinoma",
        "tissue: endometrioid adenocarcinoma"
    ]  # Extend with actual characteristics
})

# Saving cohort information
save_cohort_info('GSE94524', './preprocessed/Endometrioid_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Endometrioid_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Endometrioid_Cancer/trait_data/GSE94524.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'sample': [0], 'characteristics_ch1': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

# The identifiers in the gene expression dataset are numerical,
# which suggests they are not standard human gene symbols but likely probe IDs.
# Thus, these identifiers require mapping to gene symbols.


Index(['1', '5', '7', '9', '10', '12', '16', '17', '18', '19', '20', '22',
       '24', '25', '27', '29', '32', '35', '43', '45'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'MetaRow': ['12', '12', '12', '12', '12'], 'MetaCol': ['4', '4', '4', '4', '4'], 'SubRow': ['28', '27', '26', '25', '24'], 'SubCol': [28.0, 28.0, 28.0, 28.0, 28.0], 'Reporter ID': [334575.0, 333055.0, 331915.0, 330395.0, 328875.0], 'oligo_id': ['H300009761', 'H300009722', 'H300000470', 'H300000646', 'H300004276'], 'oligo_type': ['I', 'I', 'I', 'I', 'I'], 'gene_id': ['ENSG00000182037', 'ENSG00000180563', 'ENSG00000179449', 'ENSG00000177996', 'ENSG00000176539'], 'transcript_count': [1.0, 1.0, 1.0, 1.0, 1.0], 'representative_transcript_id': ['ENST00000315389', 'ENST00000316343', 'ENST00000314233', 'ENST00000325950', 'ENST00000326170'], 'HUGO': [nan, nan, 'MAGEL2', nan, nan], 'GB_LIST': [nan, nan, 'NM_019066, AF200625', nan, nan], 'GI-Bacillus': [nan, nan, nan, nan, nan], 'SPOT_ID': ['ENSG00000182037', 'ENSG00000180563', nan, 'ENSG00000177996', 'ENSG00000176539'], 'SEQUENCE': ['TTAATCTGACCTGTGAAAAACACTGTCCAGAGGCTAGGTGCGGTGGCTAACGCT

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Identify keys for mapping
    identifier_key = 'ID'
    gene_symbol_key = 'HUGO'

    # 2. Get the dataframe storing the mapping between probe IDs and genes
    mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # 3. Apply the mapping to the gene data
    gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Endometrioid_Cancer/gene_data/GSE94524.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Endometrioid_Cancer')
trait_biased = trait_biased

# 4. Save the cohort information.
json_path = './preprocessed/Endometrioid_Cancer/cohort_info.json'
save_cohort_info('GSE94524', json_path, True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Endometrioid_Cancer/GSE94524.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Endometrioid_Cancer':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Endometrioid_Cancer' in this dataset is fine.

